# Running TCAV


This notebook walks you through things you need to run TCAV. 

Before running this notebook, run the following to download all the data.

```
cd tcav/tcav_examples/image_models/imagenet

python download_and_make_datasets.py --source_dir=YOUR_PATH --number_of_images_per_folder=50 --number_of_random_folders=3
```

In high level, you need:

1. **example images in each folder** (you have this if you ran the above)
 * images for each concept
 * images for the class/labels of interest
 * random images that will be negative examples when learning CAVs (images that probably don't belong to any concepts)
2. **model wrapper** (below uses example from tcav/model.py)
 * an instance of  ModelWrapper abstract class (in model.py). This tells TCAV class (tcav.py) how to communicate with your model (e.g., getting internal tensors)
3. **act_generator** (below uses example from tcav/activation_generator.py)
 * an instance of ActivationGeneratorInterface that tells TCAV class how to load example data and how to get activations from the model


## Requirements

    pip install the tcav and tensorflow packages (or tensorflow-gpu if using GPU)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import os 
import tensorflow as tf
from config import root_dir, model_to_run, bottlenecks, target, concepts, version

## Step 1. Store concept and target class images to local folders

and tell TCAV where they are.

**source_dir**: where images of concepts, target class and random images (negative samples when learning CAVs) live. Each should be a sub-folder within this directory.

Note that random image directories can be in any name. In this example, we are using `random500_0`, `random500_1`,.. for an arbitrary reason. 

You need roughly 50-200 images per concept and target class (10-20 pictures also tend to work, but 200 is pretty safe).


**cav_dir**: directory to store CAVs (`None` if you don't want to store)

**target, concept**: names of the target class (that you want to investigate) and concepts (strings) - these are folder names in source_dir

**bottlenecks**: list of bottleneck names (intermediate layers in your model) that you want to use for TCAV. These names are defined in the model wrapper below.



In [3]:
# function to create project name

def list_string(list):
    str = list[0]
    for s in list[1:]:
        str += '_' + s
    return str

def create_project_name(model, bottlenecks, target, concepts, version):
    name = model_to_run
    name += '_'+ list_string(bottlenecks)
    name += ':'+ target
    name += ':'+ list_string(concepts)
    name += '_'+ str(version)
    return name

In [4]:
print ('REMEMBER TO UPDATE YOUR_PATH (where images, models are)!')
# the name of the parent directory that results are stored (only if you want to cache)
project_name = create_project_name(model_to_run, bottlenecks, target, concepts, version)
working_dir = root_dir + 'log/'  + project_name
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir+ '/activations/'
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir + '/cavs/'
# where CAVs are stored. 
tcav_dir = working_dir + '/tcavs/'
# where the images live. 
source_dir = root_dir + 'tcav/dataset/for_tcav/'
      
utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(working_dir)
utils.make_dir_if_not_exists(cav_dir)
utils.make_dir_if_not_exists(tcav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]   

print('TCAV dataset path is {}'.format(source_dir))
print('Results is saved at {}'.format(working_dir))

REMEMBER TO UPDATE YOUR_PATH (where images, models are)!
TCAV dataset path is /home/tomohiro/code/tcav/tcav/dataset/for_tcav/
Results is saved at /home/tomohiro/code/tcav/log/GoogleNet_mixed4c:zebra:dotted_striped_zigzagged_0


## Step 2. Write your model wrapper

Next step is to tell TCAV how to communicate with your model. See `model.GoogleNetWrapper_public ` for details.

You can define a subclass of ModelWrapper abstract class to do this. Let me walk you thru what each function does (tho they are pretty self-explanatory).  This wrapper includes a lot of the functions that you already have, for example, `get_prediction`.

### 1. Tensors from the graph: bottleneck tensors and ends
First, store your bottleneck tensors in `self.bottlenecks_tensors` as a dictionary. You only need bottlenecks that you are interested in running TCAV with. Similarly, fill in `self.ends` dictionary with `input`, `logit` and `prediction` tensors.

### 2. Define loss
Get your loss tensor, and assigned it to `self.loss`. This is what TCAV uses to take directional derivatives. 

While doing so, you would also want to set 
```python
self.y_input 
```
this simply is a tensorflow place holder for the target index in the logit layer (e.g., 0 index for a dog, 1 for a cat).
For multi-class classification, typically something like this works:

```python
self.y_input = tf.placeholder(tf.int64, shape=[None])
```

For example, for a multiclass classifier, something like below would work. 

```python
    # Construct gradient ops.
    with g.as_default():
      self.y_input = tf.placeholder(tf.int64, shape=[None])

      self.pred = tf.expand_dims(self.ends['prediction'][0], 0)

      self.loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
              labels=tf.one_hot(self.y_input, len(self.labels)),
              logits=self.pred))
    self._make_gradient_tensors()
```

### 3. Call _make_gradient_tensors in __init__() of your wrapper
```python
_make_gradient_tensors()  
```
does what you expect - given the loss and bottleneck tensors defined above, it adds gradient tensors.

### 4. Fill in labels, image shapes and a model name.
Get the mapping from labels (strings) to indice in the logit layer (int) in a dictionary format.

```python
def id_to_label(self, idx)
def label_to_id(self, label)
```

Set your input image shape at  `self.image_shape`


Set your model name to `self.model_name`

You are done with writing the model wrapper! I wrote two model wrapers, InceptionV3 and Googlenet.


**sess**: a tensorflow session.

In [5]:
sess = utils.create_session()

# GRAPH_PATH is where the trained model is stored.
GRAPH_PATH = source_dir + "/inception5h/tensorflow_inception_graph.pb"
# LABEL_PATH is where the labels are stored. Each line contains one class, and they are ordered with respect to their index in 
# the logit layer. (yes, id_to_label function in the model wrapper reads from this file.)
# For example, imagenet_comp_graph_label_strings.txt looks like:
# dummy                                                                                      
# kit fox
# English setter
# Siberian husky ...

LABEL_PATH = source_dir + "/inception5h/imagenet_comp_graph_label_strings.txt"

mymodel = model.GoogleNetWrapper_public(sess,
                                        GRAPH_PATH,
                                        LABEL_PATH)






You can choose ['mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e', 'mixed5a', 'mixed5b']
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## Step 3. Implement a class that returns activations (maybe with caching!)

Lastly, you will implement a class of the ActivationGenerationInterface which TCAV uses to load example data for a given concept or target, call into your model wrapper and return activations. I pulled out this logic outside of mymodel because this step often takes the longest. By making it modular, you can cache your activations and/or parallelize your computations, as I have done in `ActivationGeneratorBase.process_and_load_activations` in `activation_generator.py`.


The `process_and_load_activations` method of the activation generator must return a dictionary of activations that has concept or target name as  a first key, and the bottleneck name as a second key. So something like:

```python
{concept1: {bottleneck1: [[0.2, 0.1, ....]]},
concept2: {bottleneck1: [[0.1, 0.02, ....]]},
target1: {bottleneck1: [[0.02, 0.99, ....]]}
```


In [6]:
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=100)

## You are ready to run TCAV!

Let's do it.

**num_random_exp**: number of experiments to confirm meaningful concept direction. TCAV will search for this many folders named `random500_0`, `random500_1`, etc. You can alternatively set the `random_concepts` keyword to be a list of folders of random concepts. Run at least 10-20 for meaningful tests. 

**random_counterpart**: as well as the above, you can optionally supply a single folder with random images as the "positive set" for statistical testing. Reduces computation time at the cost of less reliable random TCAV scores. 


In [39]:
tf.logging.set_verbosity(tf.logging.INFO)
num_random_exp=1
## only running num_random_exp = 10 to save some time. The paper number are reported for 500 random runs. 
mytcav = tcav.TCAV(sess,
                   target,
                   concepts,
                   bottlenecks,
                   act_generator,
                   alphas,
                   cav_dir=cav_dir,
                   tcav_dir=tcav_dir,
                   num_random_exp=num_random_exp)#10)
print ('This may take a while... Go get coffee!')
results = mytcav.run(run_parallel=False)
print ('done!')

/home/tomohiro/code/tcav/log/GoogleNet_mixed4c:zebra:dotted_striped_zigzagged_0/logger.log
ERROR:tensorflow:the number of random concepts has to be at least 2
This may take a while... Go get coffee!
INFO:tensorflow:Running param 0 of 3
INFO:tensorflow:running zebra ['dotted', 'random500_0']
INFO:tensorflow:CAV accuracies: {'dotted': 0.9629629629629629, 'random500_0': 0.9285714285714286, 'overall': 0.9454545454545454}
INFO:tensorflow:Running param 1 of 3
INFO:tensorflow:running zebra ['striped', 'random500_0']
INFO:tensorflow:CAV accuracies: {'striped': 0.9642857142857143, 'random500_0': 1.0, 'overall': 0.9818181818181818}
INFO:tensorflow:Running param 2 of 3
INFO:tensorflow:running zebra ['zigzagged', 'random500_0']
INFO:tensorflow:CAV accuracies: {'zigzagged': 0.9629629629629629, 'random500_0': 0.9642857142857143, 'overall': 0.9636363636363636}
INFO:tensorflow:Done running 3 params. Took 7.6274638175964355 seconds...
done!


In [38]:
filename = '/home/tomohiro/code/tcav/tcav/dataset/for_tcav/zebra/zebra-card-2477671.jpg'
#filename = '/home/tomohiro/code/tcav/tcav/dataset/for_tcav/zebra/1_Zebra12778.jpg'
import numpy as np
import PIL.Image
np.array(PIL.Image.open(tf.io.gfile.GFile(filename, 'rb')))

OSError: cannot identify image file <tensorflow.python.platform.gfile.GFile object at 0x7f5b8c3f3bd0>

In [37]:
with open(filename, mode="rb") as f:
    PIL.Image.open(f.read())

OSError: [Errno 36] File name too long: b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n<title>Web Hosting, Reseller Hosting &amp; Domain Names from Heart Internet</title>\r\n<meta name="description" content="Reseller Hosting, Web Hosting, Dedicated Servers &amp; Domain Names | Heart Internet " />\r\n<meta name="keywords" content="reseller hosting, web hosting, uk, hosting, web hosts, domains, domain names, heart internet, websites, servers, dedicated servers" />\r\n<meta name="author" content="Heart Internet">\r\n<link href="http://forwards.heartinternet.uk/styles.css" rel="stylesheet" type="text/css" />\r\n</head>\r\n<body>\r\n\t<div id="container">\r\n    \t<div id="header">\r\n        \t<div id="searchbox">\r\n            \t<form id="form1" name="form1" method="get" action=\'h

In [30]:

filename = '/home/tomohiro/code/tcav/tcav/dataset/for_tcav/zebra/zebra-card-2477671.jpg'
#filename = '/home/tomohiro/code/tcav/tcav/dataset/for_tcav/zebra/1_Zebra12778.jpg'
filename = '/home/tomohiro/code/tcav/tcav/dataset/for_tcav/random500_68/180px-Orsay_proton_therapy_dsc044444959.jpg'
from io import BytesIO
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

with open(filename, 'rb') as f:
    b = BytesIO()
    f.seek(15, 0)

    b.write(f.read())

    im = Image.open(b)
    im.load()

OSError: cannot identify image file <_io.BytesIO object at 0x7f5c2c4cfad0>

In [8]:
utils_plot.plot_results(results, num_random_exp=num_random_exp)

Class = zebra
  Concept = dotted
{}


KeyError: 'mixed4c'